In [2]:
!which python

/home/stirunag/.pyenv/versions/3.10.6/bin/python


In [57]:
path_floret_model = '/home/stirunag/work/github/source_data/floret_embeddings/en_floret_model'

In [3]:
! python -m pip install floret 'spacy~=3.4.0' pandas --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-biomedner-europepmc 1.0.0 requires spacy<3.3.0,>=3.2.4, but you have spacy 3.4.4 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [58]:
import spacy

# This is the spaCy pipeline with floret vectors
nlp_fl = spacy.load(path_floret_model)

In [75]:
word_1 = nlp_fl.vocab["sars"]
word_2 = nlp_fl.vocab["sars-cove-2"]

word_1.similarity(word_2)

0.43325984477996826

In [78]:
tokens = nlp_fl("sars cov 2")
    
print(tokens.text, tokens.vector[:3], tokens.vector_norm) # Only the first three components of the vector 
    
for token in tokens:
    print(token.text, token.vector[:3], token.vector_norm)

sars cov 2 [-3.5737886  2.3306377  4.0841546] 55.919779433640024
sars [ 2.3757248  -1.0899751   0.76489997] 42.306488
cov [-1.0599601  1.1186376  3.3393645] 57.941483
2 [-12.03713   6.96325   8.1482 ] 152.65103


In [29]:
import numpy as np

def get_average_embedding(term):
    tokens = term.split()
    # Get embeddings for each token
    embeddings = [nlp_fl.vocab[token].vector for token in tokens if token in nlp_fl.vocab]
    # Compute the average embedding
    average_embedding = np.mean(embeddings, axis=0)
    return average_embedding


word_1 = nlp_fl.vocab["T2DM"]
word_2 = nlp_fl.vocab["neurofibromatosis"]
word_4 = nlp_fl.vocab["Type 2 Diabetes Mellitus T2DM"]


word_1.similarity(word_2), word_1.similarity(word_4)

(0.08433879911899567, 0.5982748866081238)

In [52]:
def get_average_embedding(term):
    tokens = term.split()
    embeddings = [nlp_fl.vocab[token].vector for token in tokens if token in nlp_fl.vocab]
    average_embedding = np.mean(embeddings, axis=0)
    return average_embedding

def cosine_similarity(vec1, vec2):
    # Compute cosine similarity between two vectors
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

word_1_vec = nlp_fl.vocab["T2DM"].vector
word_2_vec = nlp_fl.vocab["neurofibromatosis"].vector
word_3_vec = get_average_embedding("Type 2 Diabetes Mellitus T2DM")
# For word_4, we get the vector of the entire phrase
word_4_doc = nlp_fl("Type 2 Diabetes Mellitus T2DM")
word_4_vec = word_4_doc.vector

print(cosine_similarity(word_1_vec, word_2_vec))
print(cosine_similarity(word_1_vec, word_3_vec))
print(cosine_similarity(word_1_vec, word_4_vec))

0.0843388
0.5035879
0.5035879


In [36]:
word_2_vec.shape

(300,)

In [132]:
import faiss
import pickle
import spacy
import numpy as np
from fuzzywuzzy import fuzz

# Load spaCy model
nlp = spacy.load("/home/stirunag/work/github/ML_annotations/normalisation/en_floret_model")

# def get_average_embedding(term):
#     """Get the average word embedding for a term."""
#     tokens = term.split()
#     valid_vectors = [nlp.vocab[token].vector for token in tokens if
#                      nlp.vocab[token].has_vector and nlp.vocab[token].vector.shape[0] == 300]

#     if len(valid_vectors) == 0:
#         return np.zeros((300,))

#     average_embedding = np.mean(valid_vectors, axis=0)
#     return average_embedding


def get_average_embeddings_batched(terms):
    """Return average embeddings for terms."""
    docs = list(nlp.pipe(terms))
    embeddings = []

    for doc in docs:
        # Filtering out tokens without vectors or with unexpected vector sizes
        valid_vectors = [token.vector for token in doc if token.has_vector and token.vector_norm != 0 and token.vector.shape[0] == 300]

        # If no valid vectors, append a zero vector
        if len(valid_vectors) == 0:
            embeddings.append(np.zeros((300,)))
        else:
            average_embedding = np.mean(valid_vectors, axis=0)
            embeddings.append(average_embedding)

    return embeddings

# Load the term to ID mapping and indexed terms
with open("/home/stirunag/work/github/ML_annotations/normalisation/dictionary/chebi_terms.pkl", "rb") as infile:
    data = pickle.load(infile)
    term_to_id = data["term_to_id"]
    indexed_terms = data["indexed_terms"]

# Load the FAISS index
index = faiss.read_index("/home/stirunag/work/github/ML_annotations/normalisation/dictionary/chebi_terms.index")

def retrieve_similar_terms(query, k=5):
    """Retrieve top k similar terms given a query."""
    # Convert query to lowercase
    query = query.lower()
    
    # Get average embedding of the query
    query_embedding = get_average_embeddings_batched([query])
    
    norm = np.linalg.norm(query_embedding)
    query_embedding = query_embedding if norm == 0 else query_embedding / norm
    query_embedding = query_embedding.reshape(1, -1).astype('float32')

    # Search the index
    D, I = index.search(query_embedding, k)
    
    similar_terms = []
    for i in range(k):
        term = indexed_terms[I[0][i]]
        score = D[0][i]
        term_id = term_to_id[term]
        similar_terms.append((term, term_id, score))
    
    return similar_terms


def retrieve_similar_terms_with_fuzzy(query, k):
    """Retrieve k terms similar to the query."""
    query = query.lower()
    
    # Get average embedding of the query
    query_embedding = get_average_embeddings_batched([query])
    
    norm = np.linalg.norm(query_embedding)
    query_embedding = query_embedding if norm == 0 else query_embedding / norm
    query_embedding = query_embedding.reshape(1, -1).astype('float32')

    # Search the index
    D, I = index.search(query_embedding, k)
    
    # Retrieve the terms from the indexed_terms list
    candidate_terms = [indexed_terms[i] for i in I[0]]
    
    # Get fuzzy matching scores for these terms
    scores = [fuzz.ratio(query, term) for term in candidate_terms]
    
    # Pair up terms with their scores
    term_score_pairs = list(zip(candidate_terms, scores))
    
    # Rank these pairs based on scores
    ranked_term_score_pairs = sorted(term_score_pairs, key=lambda x: x[1], reverse=True)
    
    return ranked_term_score_pairs[:k]


In [130]:
# Example usage
query = "gludosulfone"
results = retrieve_similar_terms(query, 5)

for term, term_id, score in results:
    print(f"Term: {term}, ID: {term_id}, Score: {score}")

Term: fluensulfone, ID: CHEBI:131597, Score: 0.5945534110069275
Term: amidosulfuron, ID: CHEBI:2635, Score: 0.6972651481628418
Term: sulfadiasulfone, ID: CHEBI:135557, Score: 0.7089824080467224
Term: metolachlor morpholinone, ID: CHEBI:83509, Score: 0.7168866395950317
Term: dimethylvinphos, ID: CHEBI:38659, Score: 0.7179254293441772


In [147]:
# For demonstration:
query = "cholesterol"
results = retrieve_similar_terms_with_fuzzy(query, 10)
for term, score in results:
    print(f"Term: {term}, Score: {score}")

Term: cholesterol, Score: 100
Term: cholesterol_d7, Score: 88
Term: cholestane, Score: 76
Term: dihydroxycholesterol, Score: 71
Term: sterol, Score: 71
Term: oxysterol, Score: 70
Term: sitosterol, Score: 67
Term: lipoprotein cholesterol, Score: 65
Term: cholesterol glucuronide, Score: 65
Term: androstane sterol, Score: 57


In [148]:
import faiss
import pickle
import spacy
import numpy as np
from fuzzywuzzy import fuzz

# Load spaCy model
nlp = spacy.load("/home/stirunag/work/github/ML_annotations/normalisation/en_floret_model")


def get_average_embeddings_batched(terms):
    """Return average embeddings for terms."""
    docs = list(nlp.pipe(terms))
    embeddings = []

    for doc in docs:
        # Filtering out tokens without vectors or with unexpected vector sizes
        valid_vectors = [token.vector for token in doc if token.has_vector and token.vector_norm != 0 and token.vector.shape[0] == 300]

        # If no valid vectors, append a zero vector
        if len(valid_vectors) == 0:
            embeddings.append(np.zeros((300,)))
        else:
            average_embedding = np.mean(valid_vectors, axis=0)
            embeddings.append(average_embedding)

    return embeddings

# Load the term to ID mapping and indexed terms
with open("/home/stirunag/work/github/ML_annotations/normalisation/dictionary/uniprot_terms.pkl", "rb") as infile:
    data = pickle.load(infile)
    term_to_id = data["term_to_id"]
    indexed_terms = data["indexed_terms"]

# Load the FAISS index
index = faiss.read_index("/home/stirunag/work/github/ML_annotations/normalisation/dictionary/uniprot_terms.index")

def retrieve_similar_terms(query, k=5):
    """Retrieve top k similar terms given a query."""
    # Convert query to lowercase
    query = query.lower()
    
    # Get average embedding of the query
    query_embedding = get_average_embeddings_batched([query])
    
    norm = np.linalg.norm(query_embedding)
    query_embedding = query_embedding if norm == 0 else query_embedding / norm
    query_embedding = query_embedding.reshape(1, -1).astype('float32')

    # Search the index
    D, I = index.search(query_embedding, k)
    
    similar_terms = []
    for i in range(k):
        term = indexed_terms[I[0][i]]
        score = D[0][i]
        term_id = term_to_id[term]
        similar_terms.append((term, term_id, score))
    
    return similar_terms


def retrieve_similar_terms_with_fuzzy(query, k):
    """Retrieve k terms similar to the query."""
    query = query
    
    # Get average embedding of the query
    query_embedding = get_average_embeddings_batched([query])
    
    norm = np.linalg.norm(query_embedding)
    query_embedding = query_embedding if norm == 0 else query_embedding / norm
    query_embedding = query_embedding.reshape(1, -1).astype('float32')

    # Search the index
    D, I = index.search(query_embedding, k)
    
    # Retrieve the terms from the indexed_terms list
    candidate_terms = [indexed_terms[i] for i in I[0]]
    
    # Get fuzzy matching scores for these terms
    scores = [fuzz.ratio(query, term) for term in candidate_terms]
    
    # Pair up terms with their scores
    term_score_pairs = list(zip(candidate_terms, scores))
    
    # Rank these pairs based on scores
    ranked_term_score_pairs = sorted(term_score_pairs, key=lambda x: x[1], reverse=True)
    
    return ranked_term_score_pairs[:k]


In [157]:
# For demonstration:
query = "IL-1"
results = retrieve_similar_terms_with_fuzzy(query, 10)
for term, score in results:
    print(f"Term: {term}, Score: {score}")

Term: cil-1, Score: 89
Term: natSil-1, Score: 67
Term: epl-1, Score: 67
Term: blos-1, Score: 60
Term: Msil_2912, Score: 46
Term: Ccr1l1, Score: 40
Term: Meg1, Score: 25
Term: Atperox P61, Score: 13
Term: bax, Score: 0
Term: norpA, Score: 0


In [167]:
import faiss
import pickle
import spacy
import numpy as np
from fuzzywuzzy import fuzz

# Load spaCy model
nlp = spacy.load("/home/stirunag/work/github/ML_annotations/normalisation/en_floret_model")


def get_average_embeddings_batched(terms):
    """Return average embeddings for terms."""
    docs = list(nlp.pipe(terms))
    embeddings = []

    for doc in docs:
        # Filtering out tokens without vectors or with unexpected vector sizes
        valid_vectors = [token.vector for token in doc if token.has_vector and token.vector_norm != 0 and token.vector.shape[0] == 300]

        # If no valid vectors, append a zero vector
        if len(valid_vectors) == 0:
            embeddings.append(np.zeros((300,)))
        else:
            average_embedding = np.mean(valid_vectors, axis=0)
            embeddings.append(average_embedding)

    return embeddings

# Load the term to ID mapping and indexed terms
with open("/home/stirunag/work/github/ML_annotations/normalisation/dictionary/umls_terms.pkl", "rb") as infile:
    data = pickle.load(infile)
    term_to_id = data["term_to_id"]
    indexed_terms = data["indexed_terms"]

# Load the FAISS index
index = faiss.read_index("/home/stirunag/work/github/ML_annotations/normalisation/dictionary/umls_terms.index")

indexed_terms_ids = [(term, term_to_id[term]) for term in indexed_terms]

def retrieve_similar_terms(query, k=5):
    """Retrieve top k similar terms given a query."""
    # Convert query to lowercase
    query = query.lower()
    
    # Get average embedding of the query
    query_embedding = get_average_embeddings_batched([query])
    
    norm = np.linalg.norm(query_embedding)
    query_embedding = query_embedding if norm == 0 else query_embedding / norm
    query_embedding = query_embedding.reshape(1, -1).astype('float32')

    # Search the index
    D, I = index.search(query_embedding, k)
    
    similar_terms = []
    for i in range(k):
        term = indexed_terms[I[0][i]]
        score = D[0][i]
        term_id = term_to_id[term]
        similar_terms.append((term, term_id, score))
    
    return similar_terms


def retrieve_similar_terms_with_fuzzy(query, k):
    """Retrieve k terms similar to the query with their IDs."""
    query = query
    
    # Get average embedding of the query
    query_embedding = get_average_embeddings_batched([query])
    
    norm = np.linalg.norm(query_embedding)
    query_embedding = query_embedding if norm == 0 else query_embedding / norm
    query_embedding = query_embedding.reshape(1, -1).astype('float32')

    # Search the index
    D, I = index.search(query_embedding, k)
    
    # Retrieve the terms and their IDs from the indexed_terms_ids list
    candidate_terms_and_ids = [indexed_terms_ids[i] for i in I[0]]
    
    # Split into separate lists for scoring
    candidate_terms, candidate_ids = zip(*candidate_terms_and_ids)
    
    # Get fuzzy matching scores for these terms
    scores = [fuzz.ratio(query, term) for term in candidate_terms]
    
    # Pair up terms with their scores and IDs
    term_score_id_triples = list(zip(candidate_terms, scores, candidate_ids))
    
    # Rank these triples based on scores
    ranked_term_score_id_triples = sorted(term_score_id_triples, key=lambda x: x[1], reverse=True)
    
    return ranked_term_score_id_triples[:k]

In [171]:
# For demonstration:
query = "Hemophagocytosis"
results = retrieve_similar_terms_with_fuzzy(query, 10)
for term, score, term_id in results:
    print(f"Term: {term}, Score: {score}, ID: {term_id}")

Term: Hemophagocytosis, Score: 100, ID: C0876991
Term: Haemophagocytosis, Score: 97, ID: C0876991
Term: Lymphocytosis, Score: 76, ID: C0853698
Term: Hemophagocytic histiocytosis, Score: 73, ID: C5208355
Term: Hemophagocytic lymphohistiocytosis, Score: 64, ID: C0024291
Term: Hemophagocytic Lymphohistiocytosis, Score: 64, ID: C0024291
Term: Hemophagocytic Lymphohistiocytoses, Score: 64, ID: C0024291
Term: Haemophagocytic lymphohistiocytosis, Score: 63, ID: C0024291
Term: Hyperleukocytosis, Score: 61, ID: C4324336
Term: Granulocytosis, Score: 60, ID: C1282609


In [109]:
# !pip install fuzzywuzzy python-Levenshtein

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.9/172.9 kB 1.7 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 3.1 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [67]:
import faiss
import pickle
import spacy
import numpy as np

# Load spaCy model
nlp = spacy.load("/home/stirunag/work/github/ML_annotations/normalisation/en_floret_model")

def get_average_embedding(term):
    """Get the average word embedding for a term."""
    tokens = term.split()
    valid_vectors = [nlp.vocab[token].vector for token in tokens if
                     nlp.vocab[token].has_vector and nlp.vocab[token].vector.shape[0] == 300]

    if len(valid_vectors) == 0:
        return np.zeros((300,))

    average_embedding = np.mean(valid_vectors, axis=0)
    return average_embedding

# Load the term to ID mapping and indexed terms
with open("/home/stirunag/work/github/ML_annotations/normalisation/dictionary/NCBI_terms.pkl", "rb") as infile:
    data = pickle.load(infile)
    term_to_id = data["term_to_id"]
    indexed_terms = data["indexed_terms"]

# Load the FAISS index
index = faiss.read_index("/home/stirunag/work/github/ML_annotations/normalisation/dictionary/NCBI_terms.index")

def retrieve_similar_terms(query, k=5):
    """Retrieve top k similar terms given a query."""
    # Convert query to lowercase
    query = query.lower()
    
    # Get average embedding of the query
    query_embedding = get_average_embedding(query)
    norm = np.linalg.norm(query_embedding)
    query_embedding = query_embedding if norm == 0 else query_embedding / norm
    query_embedding = query_embedding.reshape(1, -1).astype('float32')

    # Search the index
    D, I = index.search(query_embedding, k)
    
    similar_terms = []
    for i in range(k):
        term = indexed_terms[I[0][i]]
        score = D[0][i]
        term_id = term_to_id[term]
        similar_terms.append((term, term_id, score))
    
    return similar_terms




In [72]:
# Example usage
query = "sars"
results = retrieve_similar_terms(query, 5)

for term, term_id, score in results:
    print(f"Term: {term}, ID: {term_id}, Score: {score}")

Term: sars, ID: 694009, Score: 0.35455653071403503
Term: sars coronavirus sin848, ID: 267387, Score: 0.5467997193336487
Term: sars coronavirus sin842, ID: 267383, Score: 0.5532567501068115
Term: sars coronavirus sin845, ID: 267395, Score: 0.5810052156448364
Term: sars coronavirus sin846, ID: 267396, Score: 0.5851927995681763


In [47]:
term_to_id['dsm 3960']

'14'

In [45]:
import faiss
import spacy
import pickle
import numpy as np

# Load the spaCy model
nlp = spacy.load("/home/stirunag/work/github/ML_annotations/normalisation/en_floret_model")

def get_average_embedding(term):
    doc = nlp(term.lower())
    valid_vectors = [token.vector for token in doc if token.has_vector and token.vector.shape[0] == 300]
    
    if len(valid_vectors) == 0:
        return np.zeros((300,))
    return np.mean(valid_vectors, axis=0)

# Load term-to-ID mapping
with open("/home/stirunag/work/github/ML_annotations/normalisation/dictionary/NCBI_terms.pkl", "rb") as infile:
    term_to_id = pickle.load(infile)

# Load the faiss index
index = faiss.read_index("/home/stirunag/work/github/ML_annotations/normalisation/dictionary/NCBI_terms.index")

def normalize_vector(vec):
    return vec / np.linalg.norm(vec)

# Create the inverse mapping
id_to_term = {v: k for k, v in term_to_id.items()}

def retrieve_similar_terms(query_term, k=5):
    # Convert query term into vector
    query_vector = normalize_vector(get_average_embedding(query_term)).reshape(1, -1).astype('float32')
    
    # Search the index for top k similar terms
    distances, indices = index.search(query_vector, k)
    
    results = []
    for i, idx in enumerate(indices[0]):
        term = id_to_term[idx]  # Use the inverse mapping
        cosine_similarity = max(0, 1 - 0.5 * distances[0][i])
        term_id = term_to_id[term]
        results.append({'term': term, 'id': term_id, 'similarity': cosine_similarity})
    
    return results

# Test the retrieval
query = "dsm 3960"
similar_terms = retrieve_similar_terms(query)
for entry in similar_terms:
    print(f"Term: {entry['term']}, ID: {entry['id']}, Similarity Score: {entry['similarity']:.3f}")


KeyError: 36

In [26]:
# term = "infectious bursal disease virus cu-1"
# vec = get_average_embedding(term)
# print(vec)
# print(np.linalg.norm(vec))

In [31]:
def get_average_embeddings_batched(terms):
    docs = list(nlp.pipe(terms))
    embeddings = []

    for doc in docs:
        valid_vectors = [token.vector for token in doc if 
                         token.has_vector and token.vector.shape[0] == 300]
        
        if len(valid_vectors) == 0:
            embeddings.append(np.zeros((300,)))
        else:
            average_embedding = np.mean(valid_vectors, axis=0)
            embeddings.append(average_embedding)

    return embeddings

# Example of usage:
terms = ["Bacteria virus cu-1", "Another example term"]  # List of terms you want to process
embeddings = get_average_embeddings_batched(terms)

In [38]:
# !pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 7.9 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [97]:
import numpy as np

def Embed(A, L):
    T = len(A)
    K = T - L + 1
    N = len(A)
    X = np.zeros((L, K))
    
    for i in range(K):
        X[:, i] = A[i:L+i]
    
    TM = np.hstack([X[:, 0].reshape(-1, 1), X])
    TM = TM.T
    
    return TM, K

# Example of use:
A = np.array([1,2,3,4,5,6,7,8,9,10])
L = 3
TM, K = Embed(A, L)
print("TM:")
print(TM[1::])
print("K:", K)


TM:
[[ 1.  2.  3.]
 [ 2.  3.  4.]
 [ 3.  4.  5.]
 [ 4.  5.  6.]
 [ 5.  6.  7.]
 [ 6.  7.  8.]
 [ 7.  8.  9.]
 [ 8.  9. 10.]]
K: 8


In [101]:
import numpy as np


def Hankelize(TM):
    N, M = TM.shape

    # Prepare an array to store summation and count of each diagonal
    summation = np.zeros(N + M - 1)
    count = np.zeros(N + M - 1)

    # Iterate over the TM matrix to populate the summation and count arrays
    for i in range(N):
        for j in range(M):
            summation[i + j] += TM[i, j]
            count[i + j] += 1

    # Element-wise division to get the average
    HM = summation / count

    return HM


# Example of use:

result = Hankelize(TM[1::])
print(result)


[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
